In [2]:
from script.scraper_functions import *
from script.soupifier import *
from serde.json import to_json

**TODO**:

- `ruff` $\Rightarrow$ linter, à utiliser plus dans le cadre du dvpt
- `pytest` avec subprocess.run pour tester le code
- `pytest-coverage`
- `poetry` avec gestion des dépendances
- `black` pour automatiquement indenter, etc.
- `doctest` pour tester les exemples des docstrings
- `bandit` pour gérer les vulnérabilités

***

- `yarl` pour travailler avec des URL

## Le code ci-dessous lance le `scraper`

- Tu peux changer le nombre d'*URL* de **19** à 1 ou 2 pour tester ça prendra beaucoup moins longtemps à générer
- Lien [`Darty`](https://www.darty.com/nav/extra/list?seller=311289-0-327183-311290-311291-328771-311452&cat=89054&state=new_offer&persist_sold_by_darty=true) $\Rightarrow$ en choisissant les produits uniquement vendus par Darty + Neufs on a 567 produits.
- Avoir un comparatif Boulanger/Darty pourrait être intéressant
- Voir pour récupérer les images des téléphones pour plus tard dans l'appli - voir : https://discuss.streamlit.io/t/add-image-and-header-to-streamlit-dataframe-table/36065/3

In [4]:
URL = "https://boulanger.com/c/nav-filtre/smartphone-telephone-portable?_etat_produit~neuf"

In [5]:
session = create_session()
url_list = create_url_list(URL, 11) #11
pages = read_pages(url_list=url_list, session=session)

Page 1 -> Successfull Extraction.
Page 2 -> Successfull Extraction.
Page 3 -> Successfull Extraction.
Page 4 -> Successfull Extraction.
Page 5 -> Successfull Extraction.
Page 6 -> Successfull Extraction.
Page 7 -> Successfull Extraction.
Page 8 -> Successfull Extraction.
Page 9 -> Successfull Extraction.
Page 10 -> Successfull Extraction.


Elapsed time for read_pages function: 39.687 seconds.

In [ ]:
all_urls = extract_all_urls(pages)
print("--------------------")
product_pages = extract_all_pages(all_urls, session)

In [ ]:
all_urls

Il faudrait créer une fonction pour la génération de la liste des smartphones + l'export en `json`

In [8]:
smartphone_list = list()
for article in product_pages:
    soup = BeautifulSoup(article, "html.parser")
    dict_features = extract_features(soup)
    smartphone = smartphone_characteristics(dict_features)
    smartphone_list.append(smartphone)

In [9]:
smartphones_json = to_json(smartphone_list)

In [10]:
file_path = "data/smartphones_2.json"

with open(file_path, "w", encoding="utf-8") as json_file:
    json_file.write(smartphones_json)

## Charge le Dataframe dans `polars`

In [1]:
import polars as pl
import polars.selectors as cs
from script.polars_gazprom import *
from itertools import chain

In [2]:
df_brut = pl.read_json("data/smartphones_2.json")

### Pipeline

Sur `535` extractions, il y a `487` téléphones valides.

In [3]:
df = NordStream(df_brut)

In [5]:
from pathlib import Path

root = Path(".").resolve()
data_folder = root / "data"
path_file = data_folder / "df_clean_2.csv"

In [3]:
df.write_csv(path_file, separator=";")

NameError: name 'df' is not defined

In [14]:
import polars as pl
import plotly.express as px

In [15]:
df = pl.read_parquet("data/sfa_results_app.parquet")

In [30]:
ctx = pl.SQLContext()

In [41]:
ctx.register("df", df).execute(
    "SELECT ram FROM df WHERE model = 'iPhone 14 Plus' AND color = 'Jaune' "
    ).collect().item()

4.0

In [31]:
df.filter(pl.col("efficiency") < 0.8).select(pl.col("model").count()).item()

85

In [9]:
df_ram = df.sort(pl.col("ram")).with_columns(
    pl.format("{} Go", pl.col("ram").cast(pl.Int64)).alias("ram_fmt")
)

fig_ram = px.box(df_ram, x="ram_fmt", y="price", points="all")
fig_ram.update_layout(yaxis=dict(title="", ticksuffix=" €"), xaxis=dict(title=""))
fig_ram.update_traces(
    boxpoints="all",
    hovertemplate="<b>Prix :</b> %{y}<br>" "<b>RAM :</b> %{x}<br>",
)

fig_ram

In [1]:
df.select(pl.corr("ram", "storage"))

NameError: name 'df' is not defined

In [78]:
fig_ram = px.box(df_ram, x="ram_fmt", y="price", points="all")

In [77]:
for trace in fig_ram.data:
    print(trace.type)

box


In [8]:
import datetime

In [12]:
current_year = datetime.date.today().year

In [27]:
df_mean_price = df.group_by("brand").agg(pl.col("price").mean().round(2)).sort("price")
df_mean_price = df_mean_price.with_columns(
    pl.format("{} €", pl.col("price")).alias("price_str")
)

least_expensive_brand = (
        df_mean_price.filter(pl.col("price") == pl.col("price").min())
        .select("brand")
        .item()
)

In [29]:
df_mean_price.filter(pl.col("price") == pl.col("price").min()).select("brand").item()

'XIAOMI'

In [30]:
def item_getter(
    df_mean_price: pl.DataFrame, column: str, min: bool = True
) -> str | float:
    if min:
        item = (
            df_mean_price.filter(pl.col("price") == pl.col("price").min())
            .select(column)
            .item()
        )
    if not min:
        item = (
            df_mean_price.filter(pl.col("price") == pl.col("price").max())
            .select(column)
            .item()
        )
    return item